In [ ]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import math as mt
import matplotlib.pyplot as plt

In [ ]:
a1=45*np.sqrt(5)
a2=24*np.sqrt(806115)
a3=1560*np.sqrt(49407661)
a4=720*np.sqrt(234602520551435)
a5=65520*np.sqrt(float(577134754173907))
a6=10080*np.sqrt(float(683295409767530975347))#

In [ ]:
def basis_fct(k,t):
    if k==1:
        return 1+0*t
    if k==2:
        return -1+np.exp(-t)
    if k==3:
        return np.exp(-t)*t
    if k==4:
        return np.exp(-t)*(1/2)*(-2*t+np.power(t,2))
    if k==5:
        return np.exp(-t)*(-6*t-36*np.power(t,2)+np.power(t,3))/a1
    if k==6:
        return np.exp(-t)*(-24*t-192*np.power(t,2)-1440*np.power(t,3)+np.power(t,4))/a2
    if k==7:
        return np.exp(-t)*(-120*t-1200*np.power(t,2)-10800*np.power(t,3)+100800*np.power(t,4)+np.power(t,5))/a3
    if k==8:
        return np.exp(-t)*(-720*t-8640*np.power(t,2)-90720*np.power(t,3)-967680*np.power(t,4)-10886400*np.power(t,5)+np.power(t,6))/a4
    if k==9:
        return np.exp(-t)*(-5040*t-70560*np.power(t,2)-846720*np.power(t,3)-10160640*np.power(t,4)-127008000*np.power(t,5)-127008000*np.power(t,6)+np.power(t,7))/a5
    if k==10:
        return np.exp(-t)*(-40320*t-645120*np.power(t,2)-8709120*np.power(t,3)-116121600*np.power(t,4)-1596672000*np.power(t,5)-1596672000*np.power(t,6)-1596671999*np.power(t,7)+np.power(t,8))/a6
    else:
        return 0

Now calculate the outputs...

In [ ]:
StartingValues=np.load('startingvaluesMultiVariatePoisson.npy')
TrainingSize=StartingValues.shape[0]
print(TrainingSize)
print(StartingValues)

In [ ]:
def evalCurve0(x,tau):
    xSize=x.size
    Y0=np.zeros(tau.size)
    for k in range(1,xSize+1,1):
        Y0+=x[k-1]*basis_fct(k,tau)
    return np.exp(Y0)


# Simulate one Brownian motion from 0 to T. We discretize daily 
T=1/12#one month simulation time horizon 
L=30#Discretization of BM
s = np.arange(0.0, 1/12, 1/(12*L))
Tminuss=T-s
print(Tminuss)
deltaS=1/(12*L)
Ssize=s.size

# [x1,..,x10] starting value x1e1 +...+ x10e10
def evalCurve(x,BMInc,tau):
    sqrtdeltaS=mt.sqrt(deltaS)
    xSize=x.size
    TauSize=tau.size
    Y0=np.zeros(TauSize)
    drift=np.zeros(TauSize)
    noise=np.zeros(TauSize)
    for k in range(1,xSize+1,1):
        Y0+=x[k-1]*basis_fct(k,tau+T)
        for j in range(0,L,1):
            tmp=basis_fct(k,tau+Tminuss[j])
            drift+=tmp*tmp*deltaS
            noise+=tmp*BMInc[k-1,j]*sqrtdeltaS
        #tmp+=x[k]*np.power(t+T,k-1)*mt.exp(-(t+T))
    return np.exp(Y0-(1/2)*drift+noise)#mt.exp(-(1/2)*T+mt.sqrt(T)*BMInc +tmp)


discretizationDimension=10#either 10 or 20
#discretize only the interval [0,1/12] as everything outside of this interval is irrelevant for the option we price
xGrid=np.linspace(0,1/12,discretizationDimension)

xValues=np.zeros([TrainingSize,discretizationDimension])

for sample in range(0,TrainingSize,1):
    xValues[sample,:]=evalCurve0(StartingValues[sample,:],xGrid)


In [ ]:
print(xValues)
np.save('DiscretizedCurve10TrainingsetMultivariatePoisson',xValues)

In [ ]:
startingValuesTestset=np.zeros([960,10])
MCprices=np.zeros([960])
for k in range(0,15,1):
    tmpstring="% s" % k
    filename1="startingvaluesMultiVariatePoissonKoefficientTestset100000Sim"+tmpstring+".npy"
    filename2="outputsMultiVariatePoissonKoefficientTestset100000Sim"+tmpstring+".npy"
    tmp1=np.load(filename1)
    tmp2=np.load(filename2)
    startingValuesTestset[k*64:(k+1)*64]=tmp1
    MCprices[k*64:(k+1)*64]=tmp2 

In [ ]:
testsize=MCprices.size
xValues=np.zeros([testsize,discretizationDimension])

for sample in range(0,testsize,1):
    xValues[sample,:]=evalCurve0(startingValuesTestset[sample,:],xGrid)



In [ ]:
print(xValues)

In [ ]:
np.save('DiscretizedCurve10TestsetMultivariatePoisson',xValues)#10 or 20

In [ ]:
np.save('outputsMultiVariatePoissonTestset100000Simulations',MCprices)